# Data Science for Business - Convolutional Neural Network (CNN) on Fashion MNIST

## Initialize notebook
Load required packages and set up workspace, e.g., initialize the random number generator.

In [ ]:
#!pip install torchmetrics

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary
from torchmetrics import Accuracy

In [ ]:
torch.manual_seed(42)

Check if we are using the GPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## Problem description

The Fashion MNIST dataset is a dataset of thumbnails of fashion articles from Zalando (28x28 grayscale images). The dataset consists of a training set of 60,000 examples and a test set of 10,000 examples. Each image is associated with a label from 10 classes (e.g., Shirts, Sneakers, Trousers...). The dataset is balanced in terms of label distribution.

You can have a look at the dataset here: https://github.com/zalandoresearch/fashion-mnist

Your task is to train a CNN classifier to predict the class of the fashion articles.

## Load data

The dataset is built into the Pytorch library, so we can load it directly.

In [4]:
batch_size = 64

We can perform a number of transformations during loading. For now, we only convert the images to Pytorch tensors.

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [6]:
train_dataset = datasets.FashionMNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root="./data", train=False, transform=transform, download=True)

In [7]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Let's check the dimensions of the training and test sets.

In [ ]:
print(train_loader.dataset.data.shape)
print(test_loader.dataset.data.shape)

Take a look at the first image in the training set.

In [ ]:
train_loader.dataset.data[0]

We can take this tensor of integers and pass it to the `imshow()` function of `matplotlib` to visualize the image. Looks like a sneaker! From Nike?

In [ ]:
plt.imshow(train_loader.dataset.data[0], cmap='gray')
plt.axis('off')
plt.show()

What is the label of this image?

In [ ]:
print("Label:", train_loader.dataset.targets[0])
print("List of labels:", train_loader.dataset.classes)


## Network architecture

Like with MLPs, we define the architecture of a CNN by creating a class that inherits form the `torch.nn.Module`.

Our simple CNN has the following architecture:
- Convolutional layer with 16 filters of size 3x3 (kernel) with a stride of 1 and padding of 1.
- ReLU activation function.
- Max pooling layer with a kernel size of 2x2 and stride of 2.
- A layer without parameters that flattens the input.
- Fully connected layer with 128 units.
- ReLU activation function.
- Fully connected layer with 10 units (one for each class).

In [12]:
# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 14 * 14, 128) # 16 out_channels of the convolution, 14x14 image dimension after pooling with kernel_size=2
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Let's initialize the CNN class and specify an optimizer and loss function. `CrossEntropyLoss` is a common loss function for multi-class classification problems.

In [13]:
learning_rate = 0.001
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
summary(model, input_size=(1, 28, 28))

# Train the network

The training loop is similar to the one for MLPs. We iterate over the training set, make predictions, calculate the loss, and update the weights of the network.

In [ ]:
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


## Make predictions

Finally, we make predictions on the test set and calculate the accuracy of the model. Don't forget to set the model to evaluation mode before making predictions.

In [ ]:
model.eval()
accuracy_metric = Accuracy(task="multiclass", num_classes=10).to(device)
accuracy_metric.reset()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Apply softmax to get probabilities
        accuracy_metric.update(probabilities, labels)

test_accuracy = accuracy_metric.compute()
print(f"Test Accuracy: {test_accuracy.item():.2f}")

We can also compute predictions for an individual image. Let's take the first image of the training set (the Nike sneaker) and see what the model predicts.

In [ ]:
logits = model(train_loader.dataset.data[0].unsqueeze(0).unsqueeze(0).float().to(device))
probs = torch.softmax(logits, dim=1)
pred = torch.argmax(probs, dim=1)
pred_label = train_loader.dataset.classes[pred]
print(probs)
print(f"Predicted label: {pred_label}")